## Fill up the configuration values 

In [0]:

#fabric workspaceid
workspace_id = ""
#fabric warehouseid
warehouse_id = ""

#read from Key vault for data security 
tenant_id = ""
service_principal_id = ""

service_principal_password = ""

configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": service_principal_id,
    "fs.azure.account.oauth2.client.secret": service_principal_password,
    "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
}




# Create a mount for OneLake
### Note: Databricks doesn’t recommend using mount points, service principals or credential passthrough; instead, Unity Catalog is suggested. This is a temporary solution until Unity catalog  OneLake direct integration is completed , creating external locations (+volumes) to OneLake are not yet possible.

In [0]:
mount_point = "/mnt/onelake-warehouse"
dbutils.fs.mount(
    source = f"abfss://{workspace_id}@onelake.dfs.fabric.microsoft.com",
    mount_point = mount_point,
    extra_configs = configs
)

### read the mirrored tables from Fabric warehouse , sample shown here for one table

In [0]:
# read with mount spn update the table name here  to your table name
df = spark.read.format("delta").load(f"/mnt/onelake-warehouse/{warehouse_id}/Tables/<tablename>/")
display(df)
df.createOrReplaceTempView("fromfabrictablenameTest")


### create a unity catalog catalog

In [0]:
%sql
-- create a new catalog
CREATE CATALOG IF NOT EXISTS fabric_catalog;

%md
### create a unity catalog Schema

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS FabrictoDBX_schema
COMMENT "A new Unity Catalog schema called FabrictoDBX_schema";

### create the table in the unity catalog schema as external table if needed, Ensure to create the external location and location credetials in unity catalog to enable external table storage 

In [0]:
%sql
--update the external storage location and the table name
CREATE OR REPLACE TABLE fabric_catalog.fabrictodbx_schema.BronzefromfabrictablenameTest
USING delta
LOCATION 'abfss://<<containername@storageaccount.dfs.core.windows.net>>/BronzefromfabrictablenameTest'
select * from fromfabrictablenameTest ;

### create a unity catalog Schema

In [0]:
%sql
--- Create a new schema in the quick_start catalog
CREATE SCHEMA IF NOT EXISTS dbxtoFabric_schema
COMMENT "A new Unity Catalog schema called dbxtoFabric_schema";

### Curate the mirrored data from fabric and create silver and gold tables

In [0]:
from pyspark.sql.functions import lit

df_silver = spark.read.format("delta").table("fabric_catalog.fabrictodbx_schema.BronzefromfabrictablenameTest")\
  .withColumn("fomdbx", lit('dbxucshortcut'))\
  .createOrReplaceTempView("silvertabletofabric")

### create a curated table from the Fabric mirrored table in the unity catalog schema as external table Ensure to create the external location and location credetials in unity catalog to enable external table storage 

In [0]:
%sql
--update the external storage location
CREATE OR REPLACE TABLE  fabric_catalog.dbxtofabric_schema.SilverTableTest
USING delta
LOCATION 'abfss://<<containername@storageaccount.dfs.core.windows.net>>/SilverfabrictablenameTest'
select * from silvertabletofabric ;